In [15]:
pip install google-api-python-client
pip install isodate

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies
import pandas as pd
import requests
import json
from matplotlib import pyplot as plt
from googleapiclient.discovery import build
import isodate

# Import the API key
from config import api_key

# Initialize YouTube Data API v3
youtube = build('youtube', 'v3', developerKey=api_key)

# Define the region codes for US(United States), GB(United Kingdom), and CA(Canada)
regions = {
    "US": "United States",
    "GB": "United Kingdom",
    "CA": "Canada"
}

def get_video_category(category_id):

    request = youtube.videoCategories().list(
        part="snippet",
        id=category_id
    )
    response = request.execute()
    category_name = response["items"][0]["snippet"]["title"]
    return category_name

def format_duration(duration):
    
    duration_obj = isodate.parse_duration(duration)
    minutes = duration_obj.total_seconds() // 60
    seconds = duration_obj.total_seconds() % 60
    return f"{int(minutes)}m {int(seconds)}s"
    
def get_trending_videos(region_code):

    # Call the API to get the top trending videos
    request = youtube.videos().list(
        part="snippet,statistics,contentDetails",
        chart="mostPopular",
        regionCode=region_code,
        maxResults=10
    )
    response = request.execute()

    # Extract video information
    videos = []
    for item in response["items"]:
        video_id = item["id"]
        video_title = item["snippet"]["title"]
        video_category_id = item["snippet"]["categoryId"]
        video_category_name = get_video_category(video_category_id)
        video_views = item["statistics"]["viewCount"]
        video_likes = item["statistics"]["likeCount"]
        video_comments = item["statistics"]["commentCount"]
        video_duration = format_duration(item["contentDetails"]["duration"])
        videos.append({
            "id": video_id,
            "title": video_title,
            "category": video_category_name,
            "views": video_views,
            "likes": video_likes,
            "comments": video_comments,
            "duration": video_duration
        })

    return videos

# Get trending videos for each region
for region_code, region_name in regions.items():
    print(f"\nTop Trending Videos in {region_name}:")
    trending_videos = get_trending_videos(region_code)
    for video in trending_videos:
        print(f"Title: {video['title']}")
        print(f"Video ID: {video['id']}")
        print(f"Category: {video['category']}")
        print(f"Views: {video['views']}")
        print(f"Likes: {video['likes']}")
        print(f"Comments: {video['comments']}")
        print(f"Duration: {video['duration']}")
        print("-" * 30)


Top Trending Videos in United States:
Title: Joker: Folie à Deux | Official Teaser Trailer
Video ID: xy8aJw1vYHo
Category: Film & Animation
Views: 14707857
Likes: 566930
Comments: 28051
Duration: 2m 25s
------------------------------
Title: PSG vs. Barcelona: Extended Highlights | UCL Quarter-Finals 1st Leg | CBS Sports Golazo
Video ID: QJOBC9O4YAk
Category: Sports
Views: 1212811
Likes: 15466
Comments: 1638
Duration: 11m 31s
------------------------------
Title: Real Madrid vs. Man. City: Extended Highlights | UCL Quarter-Finals 1st Leg | CBS Sports Golazo
Video ID: 8d4mtlZDMo4
Category: Sports
Views: 1970109
Likes: 25113
Comments: 2373
Duration: 13m 46s
------------------------------
Title: Kodak Black - Stressed Out [Official Music Video]
Video ID: 9B9eR0IOK18
Category: Music
Views: 395223
Likes: 50435
Comments: 4638
Duration: 4m 14s
------------------------------
Title: Gil's Arena GETS HONEST About Bronny's NBA Chances
Video ID: rQovYh9lMBY
Category: Sports
Views: 269181
Likes: 43

KeyError: 'likeCount'